In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger("exchangelib").setLevel(logging.WARNING)

# Connecting melusine to an Outlook Exchange mailbox

The main use-case for Melusine is **email routing**. Melusine mostly focuses on the Machine Learning aspects of email routing, however, in order to make routing effective, ML models need to be connected to a mailbox.
To connect Melusine to a mailbox, possible options are:  

**Option 1: Exposing the ML models through an API**  
Ex: An email processing system requests the Melusine API. The request contain the email content and associated metadata while the API response contain the predicted target folder for the email. Based on the API response, the email processing system is responsible for effectively moving the email in the right folder.

**Option 2: Connecting Melusine to a mailbox using a python email client**  
This way, the emails are moved to the right folders of the mailbox directly from the python code.

This tutorial demonstrates how the Melusine `ExchangeConnector` can help you with end-to-end email routing. The ExchangeConnector uses the `exchangelib` package behind the scene.  

```
>>> pip install exchangelib
```

# Routing process
The process imagined for email routing using Melusine is the following:
* Emails are received on the mailbox mymailbox@maif.fr
* Melusine is used to predict the target folder for the incoming emails
* The `ExchangeConnector` is used to move the emails to the predicted target folders

Since ML models are not perfect, some emails might be misclassified. When that happens, consumers of the mailbox are  encouraged to move the emails to the appropriate "correction folder".  
The emails in the correction folders will constitute training data for future model trainings and thus improve the model.  


# The ExchangeConnector

The Melusine `ExchangeConnector` is instanciated with the following arguments:
* `login_address`: Email address used to login and send emails
* `password`: The password associated with the mailbox address
* `mailbox_address`: Email address of the mailbox (ex: mymailbox@maif.fr). By default, the login address is used 
* `routing_folder_path`: Path to the folder that contains the routed emails
* `correction_folder_path`: Path to the folder that contains the corrected emails
* `done_folder_path`: Path to the folder that contains "Done" emails (emails that have already been processed)
* `max_wait`: The maximum time to be spent on trying to connect to the mailbox. Once this time is over, a connection error is raised
* `target_column`: When routing, name of the DataFrame column containing target folders"target" (Default: target)

In [ ]:
my_login_address = "myuser@maif.fr"
my_password = "melusineisawesome"

# Assuming the mailbox address is the same as the login address
my_mailbox_address = my_login_address

In [ ]:
from melusine.connectors.exchange import ExchangeConnector
connector = ExchangeConnector(
    login_address=my_login_address, 
    password=my_password, 
    mailbox_address=my_mailbox_address, 
    max_wait=60
)

# Send fake emails
In this section a set of fake emails are sent to the mailbox.  
We will then use Melusine and the `ExchangeConnector` to route these emails.

## Send emails
The `send_email` method is used to send emails.  

In [ ]:
fake_emails = [
    {
        "header": "[Melusine Test]",
        "body": "This should go to folder Test1"
    },
    {
        "header": "[Melusine Test]",
        "body": "This should go to folder Test2"
    },
    {
        "header": "[Melusine Test]",
        "body": "This should go to folder Test3"
    }
]

In [ ]:
for email_dict in fake_emails:
    connector.send_email(
        to=[my_mailbox_address], 
        heheader=email_dict["header"], 
        body=email_dict["body"], 
        attachments=None
)

**Expected output:**  
You should receive 3 emails in your mailbox

# Create folders
In the email routing scenario considered, the following folders are needed:  

**Target folders**  
These are the folders where the routed emails will be stored.
* `Inbox / ROUTING / Test1`
* `Inbox / ROUTING / Test2`
* `Inbox / ROUTING / Test3`

**Correction folders**  
When an email is erroneously routed to a target folder, mailbox consumers can move the email to the appropriate "Correction folder".  
* `Inbox / CORRECTION / Test1`
* `Inbox / CORRECTION / Test2`
* `Inbox / CORRECTION / Test3`

**Done folder**
Once the emails in the correction folders have been processed (ex: for model re-training), the correction folders can be flushed by moving all the emails in the Done folder.  
* `Inbox / DONE`

## Setup ROUTING folder structure

In [ ]:
# Print path to the default routing folder (We will update it later)
f"Default ROUTING folder path : '{connector.routing_folder_path}'"

In [ ]:
# Create the base routing folder
connector.create_folders(["ROUTING"], base_folder_path=None)

In [ ]:
# Create the routing subfolders
connector.create_folders(["Test1", "Test2", "Test3"], base_folder_path="ROUTING")

In [ ]:
# Setup the routing folder path
connector.routing_folder_path = "ROUTING"
f"Updated ROUTING folder path :'{connector.routing_folder_path}'"

In [ ]:
# Print folder structure
print(connector.routing_folder.tree())

**Expected output:** 
<pre>
ROUTING  
├── Test1
├── Test2
└── Test3
</pre>

## Setup the CORRECTION folder structure

In [ ]:
f"Default CORRECTION folder path :'{connector.correction_folder_path}'"

In [ ]:
# Create the base CORRECTION folder at the inbox root
connector.create_folders(["CORRECTION"], base_folder_path=None)

In [ ]:
# Create the correction subfolders
connector.create_folders(["Test1", "Test2", "Test3"], base_folder_path="CORRECTION")

In [ ]:
# Setup the correction folder path
connector.correction_folder_path = "CORRECTION"
f"Updated CORRECTION folder path :'{connector.correction_folder_path}'"

In [ ]:
# Print folder structure
print(connector.correction_folder.tree())

**Expected output:** 
<pre>
CORRECTION  
├── Test1
├── Test2
└── Test3
</pre>

## Setup the DONE folder

In [ ]:
# Create the DONE folder at the inbox root
connector.create_folders(["DONE"], base_folder_path=None)

In [ ]:
# Setup the done folder path
connector.done_folder_path = "DONE"
f"Updated DONE folder path :'{connector.done_folder_path}'"

In [ ]:
# Print folder structure
print(connector.mailbox_account.inbox.tree())

**Expected output:** 
<pre>
Boîte de réception
├── ROUTING
│   ├── Test1
│   ├── Test2
│   └── Test3
├── CORRECTION
│   ├── Test1
│   ├── Test2
│   └── Test3
└── DONE
</pre>

# Load emails
Before emails can be routed, we need to load the content of new emails.  
The `get_emails` method loads the content of a mailbox folder (by default: the inbox folder).

In [ ]:
df_emails = connector.get_emails(max_emails=50, ascending=False)

In [ ]:
# Pick only test emails
mask = df_emails["header"] == "[Melusine Test]"
df_emails = df_emails[mask].copy()

# reverse order
df_emails = df_emails.reindex(index=df_emails.index[::-1])

df_emails.drop(["message_id"], axis=1)

**Expected output:**

|    | message_id                                                                      | body     | header          | date                      | from                         | to                               | attachment   |
|---:|:--------------------------------------------------------------------------------|:---------|:----------------|:--------------------------|:-----------------------------|:---------------------------------|:-------------|
| 61 | <1> | This should go to folder Test1 | [Melusine Test] | 2021-05-04T19:07:56+00:00 | mymailbox@maif.fr | ['mymailbox@maif.fr'] |              |
| 62 | <2> | This should go to folder Test2 | [Melusine Test] | 2021-05-04T19:07:55+00:00 | mymailbox@maif.fr | ['mymailbox@maif.fr'] |              |
| 63 | <3> | This should go to folder Test3 | [Melusine Test] | 2021-05-04T19:07:56+00:00 | mymailbox@maif.fr | ['mymailbox@maif.fr'] |              |

# Predict target folders
This tutorial focuses on the exchange connector so the ML model prediction part is mocked. Feel free to check the `tutorial08_full_pipeline_detailed.ipynb` to see how ML predictions work with Melusine.

In [ ]:
def fake_predictions(emails):
    predictions = []
    for i in range(len(emails)):
        predictions.append(f"Test{i%3+1}")
    
    # Introduce a missclassification
    predictions[0] = "Test2"
    
    emails["target"] = predictions
    return emails

df_emails = fake_predictions(df_emails)
df_emails[["header", "body", "target"]]

**Expected output:**

|    | header          | body                           | target   |
|---:|:----------------|:-------------------------------|:---------|
| 76 | [Melusine Test] | This should go to folder Test1 | Test2    |
| 77 | [Melusine Test] | This should go to folder Test2 | Test2    |
| 78 | [Melusine Test] | This should go to folder Test3 | Test3    |

As you can see, there is a prediction error for the first email (`Test2` instead of `Test1`)

# Route emails
Now that we have predicted the target folders for each email, we use the `ExchangeConnector` to move the emails in the mailbox.  
The `route_emails` does exactly that. Its argument are:  
        classified_emails,
        on_error="warning",
        id_column="message_id",
        target_column="target",
* `classified_emails`: The DataFrame containing the emails and their predicted target folder
* `raise_missing_folder_error`: If activated, an error is raised when the target folder does not exist in the mailbox. Otherwise, a warning is printed and the emails are left in the inbox.
* `id_column`: Name of the DataFrame column containing the message ID
* `target_column`: Name of the DataFrame column containing the target folder

In [ ]:
connector.route_emails(df_emails)

In [ ]:
connector.get_emails(base_folder_path="ROUTING/Test2")[["header", "body"]]

**Expected output:**

|    | message_id                                                                      | body     | header          | date                      | from                         | to                               | attachment   |
|---:|:--------------------------------------------------------------------------------|:---------|:----------------|:--------------------------|:-----------------------------|:---------------------------------|:-------------|
| 61 | <1> | This should go to folder Test1 | [Melusine Test] | 2021-05-04T19:07:56+00:00 | mymailbox@maif.fr | ['mymailbox@maif.fr'] |              |
| 62 | <2> | This should go to folder Test2 | [Melusine Test] | 2021-05-04T19:07:55+00:00 | mymailbox@maif.fr | ['mymailbox@maif.fr'] |              |

Two emails have been routed to the folder `Test2` !

# Make CORRECTION
## Move emails to correction folders
CORRECTION should be made by the mailbox consumers directly in the mailbox.   


Go to your mailbox and move the emails that says:  
**"This should go to folder Test1"**  
(currently in the Test2 folder)
To the correction folder `CORRECTION/Test1`

## Load corrected data

In [ ]:
df_CORRECTION = connector.get_CORRECTION()
df_CORRECTION

**Expected output:**

|    | message_id                                                                      | body     | header          | date                      | from                         | to                               | attachment   |
|---:|:--------------------------------------------------------------------------------|:---------|:----------------|:--------------------------|:-----------------------------|:---------------------------------|:-------------|
| 61 | <1> | This should go to folder Test1 | [Melusine Test] | 2021-05-04T19:07:56+00:00 | mymailbox@maif.fr | ['mymailbox@maif.fr'] |              |

The emails loaded from the correction folder can now be used to train a new ML model !

# Move corrected emails to the "Done" folder

In [ ]:
connector.move_to_done(df_CORRECTION["message_id"])

# Conclusion
With the `ExchangeConnector` you should be able to easily implement email routing for your mailbox using Melusine !   


**Hint :** If you like Melusine, don't forget to add a star on [GitHub](https://github.com/MAIF/melusine)